In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
import requests

import pandas as pd
import numpy as np

import re

Here, I scrape passenger itinerary data from the [Bureau of Transport Statistics](https://transtats.bts.gov). These data are available on a quarterly basis in .zip files containing .csv files, with records for 10% of passenger itineraries on US flights. These data include origin, destination, fare, fare class, and the different segments flown for a given itinerary. I use Selenium because there are links to be clicked and multiple pages through which to navigate.

In [16]:
chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get('https://transtats.bts.gov/')

In [17]:
buttons = driver.find_elements_by_xpath('//td[@class="sidebarLink"]')

In [18]:
for button in buttons:
    if button.find_element_by_tag_name('a').text == 'Aviation':
        button.find_element_by_tag_name('a').click()
        break

In [19]:
for link in driver.find_elements_by_xpath('//td[@class="dataTD"]/a'):
    if link.text == 'Airline Origin and Destination Survey (DB1B)':
        link.click()
        break

The code below might break even after it successfully scrapes files -- for example, the indices specified might not have a file for a given year (such as the current year where there will only be records available for the first quarter or two).

In [20]:
links = len(driver.find_elements_by_xpath('//td[@class="dataTDRight"]/a'))
for link in range(links):
    if driver.find_elements_by_xpath('//td[@class="dataTDRight"]/a')[link].text == 'Download':
        driver.find_elements_by_xpath('//td[@class="dataTDRight"]/a')[link].click()
        #time.sleep(30)
        driver.find_element_by_xpath('//input[@name="DownloadZip"]').click()
        for year in driver.find_elements_by_xpath('//select[@id="XYEAR"]/option')[23:]:
            driver.find_element_by_xpath('//select[@id="XYEAR"]').click()
            year.click()
            for qtr in driver.find_elements_by_xpath('//select[@id="FREQUENCY"]/option'):
                qtr.find_element_by_xpath('//select[@id="FREQUENCY"]')
                qtr.click()
                driver.find_element_by_xpath('//button[@name="Download2"]').click()
        driver.back()

UnexpectedAlertPresentException: Alert Text: None
Message: unexpected alert open: {Alert text : Requested data do not exist.}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.36.540469 (1881fd7f8641508feb5166b7cae561d87723cfa8),platform=Mac OS X 10.13.1 x86_64)


In [ ]:
driver.quit()
driver.close()

The code belows unzips all the files scraped from above and saves them into a folder with a specified path (this can be modified to your default download directory, as that is where the files from above are saved).

In [2]:
import zipfile
from os.path import expanduser
home = expanduser("~")

In [3]:
def unzip_all_db1b_coup(years, qtrs):
    for yr in years:
        for qtr in qtrs:
            path = home + '/Downloads/Origin_and_Destination_Survey_DB1BCoupon_' + str(yr) + '_' + str(qtr) + '.zip'
            with zipfile.ZipFile(path,'r') as zip_ref:
                zip_ref.extractall('DB1BCoupon')

In [7]:
years = np.arange(2008, 2018)
qtrs = np.arange(1, 5)
unzip_all_db1b_coup(years, qtrs)

In [4]:
unzip_all_db1b_coup([2018], [1])